<a href="https://colab.research.google.com/github/EdWangLoDaSc/DVI_BNN/blob/main/BNN_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
sys.path.append('../../')
import torch_ard as nn_ard
from torch import nn
import torch.nn.functional as F
import torch

class DenseModelARD(nn.Module):
    def __init__(self, input_shape, output_shape, hidden_size=150, activation=None):
        super(DenseModelARD, self).__init__()
        self.l1 = nn_ard.LinearARD(input_shape, hidden_size)
        self.l2 = nn_ard.LinearARD(hidden_size, output_shape)
        self.activation = activation
        self._init_weights()

    def forward(self, input):
        x = input.to(self.device)
        x = self.l1(x)
        x = nn.functional.tanh(x)
        x = self.l2(x)
        if self.activation: x = self.activation(x)
        return x

    def _init_weights(self):
        for layer in self.children():
            if hasattr(layer, 'weight'): nn.init.xavier_uniform(layer.weight, gain=nn.init.calculate_gain('relu'))

    @property
    def device(self):
        return next(self.parameters()).device


class DenseModel(nn.Module):
    def __init__(self, input_shape, output_shape, hidden_size=150, activation=None):
        super(DenseModel, self).__init__()
        self.l1 = nn.Linear(input_shape, hidden_size)
        self.l2 = nn.Linear(hidden_size, output_shape)
        self.activation = activation
        self._init_weights()

    def forward(self, input):
        x = input.to(self.device)
        x = self.l1(x)
        x = nn.functional.tanh(x)
        x = self.l2(x)
        if self.activation: x = self.activation(x)
        return x

    def _init_weights(self):
        for layer in self.children():
            if hasattr(layer, 'weight'): nn.init.xavier_uniform(layer.weight, gain=nn.init.calculate_gain('relu'))

    @property
    def device(self):
        return next(self.parameters()).device

class LeNet(nn.Module):
    def __init__(self, input_shape, output_shape):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(input_shape, 20, 5)
        self.conv2 = nn.Conv2d(20, 50, 5)
        self.l1 = nn.Linear(50*5*5, 500)
        self.l2 = nn.Linear(500, output_shape)
        self._init_weights()

    def forward(self, x):
        out = F.relu(self.conv1(x.to(self.device)))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.shape[0], -1)
        out = F.relu(self.l1(out))
        return self.l2(out)
        # return F.log_softmax(self.l2(out), dim=1)

    def _init_weights(self):
        for layer in self.children():
            if hasattr(layer, 'weight'): nn.init.xavier_uniform(layer.weight, gain=nn.init.calculate_gain('relu'))

    @property
    def device(self):
        return next(self.parameters()).device

class LeNetARD(nn.Module):
    def __init__(self, input_shape, output_shape):
        super(LeNetARD, self).__init__()
        self.conv1 = nn_ard.Conv2dARD(input_shape, 20, 5)
        self.conv2 = nn_ard.Conv2dARD(20, 50, 5)
        self.l1 = nn_ard.LinearARD(50*5*5, 500)
        self.l2 = nn_ard.LinearARD(500, output_shape)
        self._init_weights()

    def forward(self, input):
        out = F.relu(self.conv1(input.to(self.device)))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.shape[0], -1)
        out = F.relu(self.l1(out))
        return self.l2(out)
        # return F.log_softmax(self.l2(out), dim=1)

    def _init_weights(self):
        for layer in self.children():
            if hasattr(layer, 'weight'): nn.init.xavier_uniform(layer.weight, gain=nn.init.calculate_gain('relu'))

    @property
    def device(self):
        return next(self.parameters()).device

ModuleNotFoundError: ignored

In [ ]:
import sys
sys.path.append('../')
from models import DenseModel
from sklearn.model_selection import train_test_split
import pandas as pd
from torch import nn
import torch
import numpy as np
from tqdm import tqdm, trange

boston = load_boston()
df = pd.read_csv('3_256_datsets.csv')

X, y = df.drop('PRICE', 1), df['PRICE']

train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.8)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_X, test_X, train_y, test_y = \
    [torch.from_numpy(np.array(x)).float().to(device)
     for x in [train_X, test_X, train_y, test_y]]

model = DenseModel(input_shape=train_X.shape[1], output_shape=1,
                   activation=nn.functional.relu).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, 'min')
criterion = nn.MSELoss()

n_epoches = 100000
debug_frequency = 100

pbar = trange(n_epoches, leave=True, position=0)
for epoch in pbar:
    opt.zero_grad()
    preds = model(train_X).squeeze()
    loss = criterion(preds, train_y)
    loss.backward()
    # scheduler.step(loss)
    opt.step()
    loss_train = float(criterion(preds, train_y).detach().cpu().numpy())
    preds = model(test_X).squeeze()
    loss_test = float(criterion(preds, test_y).detach().cpu().numpy())
    pbar.set_description('MSE (train): %.3f\tMSE (test): %.3f' %
                         (loss_train, loss_test))
    pbar.update()

In [ ]:
import sys
sys.path.append('../')
from models import DenseModelARD
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import pandas as pd
from torch import nn
import torch.nn.functional as F
import torch
import numpy as np
from torch_ard import get_ard_reg, get_dropped_params_ratio, ELBOLoss
from tqdm import trange, tqdm

boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['PRICE'] = boston.target
X, y = df.drop('PRICE', 1), df['PRICE']

train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.8)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')
train_X, test_X, train_y, test_y = \
    [torch.from_numpy(np.array(x)).float().to(device)
     for x in [train_X, test_X, train_y, test_y]]

model = DenseModelARD(input_shape=train_X.shape[1], output_shape=1,
                      activation=nn.functional.relu).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, 'min')
criterion = ELBOLoss(model, F.mse_loss).to(device)

n_epoches = 100000
debug_frequency = 100
def get_kl_weight(epoch): 
  return min(1, 2 * epoch / n_epoches)


pbar = trange(n_epoches, leave=True, position=0)
for epoch in pbar:
    kl_weight = get_kl_weight(epoch)
    opt.zero_grad()
    preds = model(train_X).squeeze()
    loss = criterion(preds, train_y, 1, kl_weight)
    loss.backward()
    opt.step()
    loss_train = float(
        criterion(preds, train_y, 1, 0).detach().cpu().numpy())
    preds = model(test_X).squeeze()
    loss_test = float(
        criterion(preds, test_y, 1, 0).detach().cpu().numpy())
    pbar.set_description('MSE (train): %.3f\tMSE (test): %.3f\tReg: %.3f\tDropout rate: %f%%' % (
        loss_train, loss_test, get_ard_reg(model).item(), 100 * get_dropped_params_ratio(model)))
    pbar.update()
Footer
